<p style='
  color: #3b4045; 
  text-align: center;
  font-weight: bold;
  font-family: -apple-system,BlinkMacSystemFont, "Segoe UI Adjusted","Segoe UI","Liberation Sans",sans-serif;     font-size: 2.07692308rem; '> 
    Vascular Graph Extraction
</p>


In [1]:
from fundus_vessels_toolkit.seg2graph import RetinalVesselSeg2Graph

In [2]:
# Path to the raw fundus image
RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/downsampled-1024/1-images/100_A.jpg"

# Path to a vessel ground truth (for comparison purposes)
VESSELS_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/downsampled-1024/2-vessels/100_A.png"

### Load the raw Image and the Vascular Segmentation


In [3]:
import cv2


def load_img(path, binarize=False):
    img = cv2.imread(path)
    img = img.astype(float) / 255
    return img.mean(axis=2) > 0.5 if binarize else img

Load image and segmentation mask


In [6]:
raw = load_img(RAW_PATH)
vessels = load_img(VESSELS_PATH, binarize=True)

Perform segmentation using a pretrained model


In [5]:
from fundus_vessels_toolkit.models import segment

vessels = segment(raw)

ModuleNotFoundError: No module named 'timm.models.layers.activations'

---


### Skeletonize and Extract Graph


All the skeletonization and graph extraction pipeline is accessible through `RetinalVesselSeg2Graph` objects.

This object tunes all the parameters of the pipeline according to a single parameter: the maximum vessel diameter.
Apriori, a good approximation is a 80th of the image width (13px for a 1024x1024 px image).


In [8]:
max_vessel_diameter = raw.shape[1] // 80
seg2graph = RetinalVesselSeg2Graph(max_vessel_diameter)

To compute the graph extraction you simply need to call the object with a segmentation map:


In [10]:
vgraph = seg2graph(vessels)
true_vgraph = seg2graph(vessels)

The returned object is a `VascularGraph` which expose all the properties of the extracted graph.

- Boolean matrix encoding the connection between branches (rows) and nodes (columns): `vgrapp.branch_by_node`
- The y and x position of the nodes: `vgraph.nodes_yx_coord`
- The label map of branches: `vgraph.branch_labels_map`

This object is also usefull to cast the graph to other representation. For example:

- Node Adjacency Matrix: `vgraph.node_adjacency_matrix()`
- Branch Adjacency Matrix: `vgraph.branch_adjacency_matrix()`


#### Display


Lets compare the graph extracted on the predicted segmentation and on the ground truth segmentation side by side!

_(The prediction is on the left, the ground truth on the right.)_


In [12]:
import numpy as np
from jppype.view2d import imshow, View2D, sync_views
from ipywidgets import GridspecLayout


def create_view(vessels, vgraph):
    v = imshow(raw)
    v.add_label(vessels, "vessel", "white", options={"opacity": 0.2})
    v["vessel graph"] = vgraph.jppype_layer(edge_map=True)

    return v


grid = GridspecLayout(1, 2, height="650px")
grid[0, 0] = create_view(vessels, vgraph)
grid[0, 1] = create_view(vessels, true_vgraph)
sync_views(grid[0, 0], grid[0, 1])

grid

GridspecLayout(children=(View2D(layout=Layout(grid_area='widget001'), linkedTransformGroup='240cd52691184a64a0…

---


### Performing only the skeletonization


The method `RetinalVesselSeg2Graph.seg2adjacency()` actually perform the graph extraction in two steps:

- The skeletonization (morphological reduction of the segmentation to the vessels centerline): `seg2graph.skeletonize(vessel_segmentation_map)`
- The graph extraction (identification of branches and nodes in the vascular skeleton and connection parsing): `seg2graph.skel2adjacency(vessel_skeleton_map)`.

Those method can be called individually, for example to get the result of the skeletonization step only.


In [ ]:
skeleton = seg2graph.skeletonize(vessels)

You may notice some difference between the skeleton obtained through `seg2graph.skeletonize(vessels)` and by direct call of `seg2graph(vessels)`, because several topological simplification is performed on the latter (merging close nodes, removing small spurs or cycles...).


In [ ]:
v = imshow(raw)
v.add_label(vessels, "vessel", "white", options={"opacity": 0.2})
v.add_label(skeleton, "skeleton", {2: "white", 1: "yellow", 3: "red", 4: "purple"})
v

View2D()

<br />
<hr style="width:20%;text-align:center;"/>

Calling `seg2graph.seg2adjacency(vessel_map)` will return the adjacency matrix `adj` which store the graph links between branches (rows) and nodes (columns).

If `return_label=True` this method also returns:

- the label map of branches where the value of each pixel correspond to its branch id **+ 1** (or 0 if the pixel is not part of the skeleton);
- the coordinates of each nodes as a tuple (y, x) where y and x are vectors of length `adj.shape[1]`.


In [ ]:
adj, branch_labels, node_yx = seg2graph.seg2adjacency(vessels, return_label=True)

In [ ]:
v = imshow(raw)
v.add_label(branch_labels)
v

View2D()